In [1]:
# My Google Drive Mount하기!
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Random Search

In [2]:
import numpy as np
import pandas as pd

In [3]:
x = np.random.normal(
    size=(1000, 20)
)
print(x.shape)

(1000, 20)


In [4]:
print(x)

[[-1.03598303 -2.30208425 -0.25174291 ... -0.49817488 -0.33994825
  -0.69988844]
 [-0.4655364   1.07709763  0.70012382 ... -0.17860304 -0.43753825
   1.36303077]
 [ 1.81076928  0.61066978 -0.89377251 ...  1.39444701 -0.49945511
   0.83785487]
 ...
 [-0.27203056  1.01034539 -1.12898628 ... -0.67562134  0.47351776
   0.11401485]
 [-1.23509024 -0.86412144  0.05308727 ...  2.4126968   0.57258658
   1.59059638]
 [ 0.09801797  0.28682805  1.60978113 ... -2.8332125   0.09250784
   0.47880236]]


In [5]:
y = x.sum(axis=1)

y = np.where(
    np.abs(y) > 1,
    1,
    0
)

print(y.shape)

(1000,)


In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
    train_test_split(
        x,
        y,
        test_size=0.2,
        random_state=1234
    )

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(800, 20) (800,)
(200, 20) (200,)


* Model을 만들고 학습시키는 과정을 User Defined Function으로 작성한다.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

FolderPath = '/content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data'

def CreateModelFitModelGetMaxValAcc(idx, Node_cnt, HiddenLayer_cnt, act, lr):

    model = Sequential()
    for _ in range(HiddenLayer_cnt):
        model.add(Dense(Node_cnt, activation=act))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['acc']
    )

    CKP = ModelCheckpoint(
        filepath=FolderPath + '/' + f'model{idx:02d}',
        monitor='val_acc',
        save_best_only=True
    )

    history = model.fit(
        train_input,
        train_target,
        epochs=10,
        validation_split=0.2,
        callbacks=[CKP],
        verbose=0
    )

    return max(history.history['val_acc'])

* Random Search

In [8]:
result_list = []

for idx in range(3):

    Node_cnt_list = [2, 4, 6, 8]
    Node_cnt = np.random.choice(Node_cnt_list)

    HiddenLayer_cnt = np.random.randint(0, 3)
    Act = np.random.choice(['relu', 'tanh'])
    LR = 10 ** np.random.randint(-5, 0)

    val_acc = CreateModelFitModelGetMaxValAcc(idx, Node_cnt, HiddenLayer_cnt, Act, LR)

    result_list.append({
        'Val_acc': val_acc,
        'Node_cnt': Node_cnt,
        'HiddenLayer_cnt': HiddenLayer_cnt,
        'Activation': Act,
        'Learning Rate': LR
    })

INFO:tensorflow:Assets written to: /content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data/model00/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data/model01/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data/model01/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data/model01/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data/model01/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data A

* 가장 좋은 모델 불러오기

In [9]:
pd.DataFrame(result_list)

,Val_acc,Node_cnt,HiddenLayer_cnt,Activation,Learning Rate
0,0.50000,4,0,tanh,0.00010
1,0.76250,4,2,relu,0.00100
2,0.50625,6,0,relu,0.00001


In [11]:
from tensorflow.keras.models import load_model

model = load_model(FolderPath + '/' + 'model01')

* Test

In [12]:
model.evaluate(test_input, test_target)

7/7 [==============================] - 0s 2ms/step - loss: 0.5467 - acc: 0.7800


[0.5466504693031311, 0.7799999713897705]